# Acceso a bases de datos

## Open Database Connectivity (ODBC)

Es un estándar de acceso a bases de datos que permite que aplicaciones puedan conectarse a diferentes SGBD independientes (interoperabilidad), reduciendo las dependencias del proveedor.

* Proporciona una __interfaz común__ para que las aplicaciones accedan a diferentes tipos de bases de datos.
* Actúa como un ___puente___ entre la aplicación y el SGBD.
* Permite cambiar el motor de base de datos sin modificar la lógica de la aplicación, siempre que se mantenga la estructura de datos.
* Su uso requiere configuración del driver y del DSN.

## Funcionamiento general

1. El cliente realiza las peticiones en SQL.
2. El driver ODBC traduce las peticiones al lenguaje específico del SGBD.
3. El SGBD procesa la consulta y devuelve el resultado.


In [4]:
from IPython.display import Image
Image(url="https://dev.mysql.com/doc/connectors/en/images/myarchitecture.png", width=400, retina=True)

_Arquitectura de conector ODBC.
Fuente: [MySQL.com](https://dev.mysql.com/doc/connector-odbc/en/connector-odbc-architecture.html)_

### Aplicaciones

- Aplicaciones utilizan la API ODBC para acceder a los datos en el servidor MySQL.
- La API ODBC a su vez se comunica con el Administrador de Controladores.
- La Aplicación se comunica con el Administrador de Controladores utilizando las llamadas ODBC.

### Administrador de Controladores
Es una biblioteca que gestiona la comunicación entre las aplicaciones y los controladores. Realiza las siguientes tareas:

- Resuelve nombres de fuentes de datos (DSN).
- Requiere cargar y descargar el controlador específico para acceder a una base de datos según se define en el DSN.
- Procesa llamadas a funciones ODBC o las pasa al controlador para su procesamiento.

### Controlador Connector/ODBC

Es una biblioteca que implementa las funciones admitidas por la API ODBC.
- Procesa llamadas a funciones ODBC,
- Envía solicitudes SQL al servidor MySQL y devuelve los resultados a la aplicación.
- Si es necesario, el controlador modifica la solicitud de una aplicación para que la solicitud se ajuste a la sintaxis compatible con MySQL.

### Servidor MySQL

- Corresponde a la base de datos MySQL donde se almacena la información.

### Configuración de DSN

- El archivo de configuración ODBC almacena la información del controlador y la base de datos necesaria para conectarse al servidor.
- El Administrador de Controladores lo utiliza para determinar qué controlador se debe cargar de acuerdo con la definición en el DSN.
- La configuración de conexión (DSN - *Data Source Name*) [[1](https://support.microsoft.com/en-us/help/966849/what-is-a-dsn-data-source-name)][[2](https://en.wikipedia.org/wiki/Data_source_name)] permite establecer la comunicación con la fuente de datos ODBC, es decir, establece los detalles de conexión: el nombre de la base de datos, el directorio, el controlador de la base de datos, identificación de usuario y contraseña la contraseña, entre otros.

## Instalación del driver ODBC y configuración de la conexión

__IMPORTANTE__: Para establecer una concexión, debe conocer los parámetros de conexión. Si trabaja con un servidor en su propia máquina (local), debe contar con una instalación del motor de base de datos, de lo contrario, puede instalar MySQL a partir de una de las siguientes alternativas,

- Instalar [MySQL](https://dev.mysql.com/downloads/workbench/).
- Instalar MySQL por medio del paquete [XAMPP](https://www.apachefriends.org/es/index.html).

Para instalar el conector ODBC y configuración de la conexión,

1. Descargar e instalar el [Connector/ODBC de MySQL para Windows](https://dev.mysql.com/downloads/connector/odbc/).
2. Configurar conexión a una base de datos existente en MySQL ([instrucciones](https://dev.mysql.com/doc/connector-odbc/en/connector-odbc-configuration-dsn-windows-5-2.html)). Para esto debe conocer los parámetros de configuración de su servidor de base de datos.
    - **Server**: Nombre del Host o dirección IP del servidor MySQL. Por defecto es `localhost`.
    - **User**: Nombre de usuario de MySQL para conectarse al servidor MySQL. Habitualmente es `root`.
    - **Password**: Contraseña asociada al usuario __User__.
    - **Database**: Nombre de la base de datos a la cual se conectará una vez que se inicie la conexión. Se puede selección desde el menú desplegable.

## Conexión ODBC desde Python

[Pyodbc](https://pypi.org/project/pyodbc/#description) es un módulo Python de código abierto que simplifica el acceso a las bases de datos ODBC. Implementa la especificación DB API 2.0.

* Debe tener instalado un administrador de controladores ODBC antes de instalar pyodbc.
* En Windows, el [administrador de controladores](https://learn.microsoft.com/en-us/sql/database-engine/configure-windows/open-the-odbc-data-source-administrator?view=sql-server-ver17) ODBC está integrado.

## MySQL connector para Python

- [`MySQL connector`](https://dev.mysql.com/doc/connector-python/en/) es una API que cumple con especificación Python Database API (PEP 249).
- [PEP 249](https://www.python.org/dev/peps/pep-0249/) fomenta la similitud entre los módulos de Python que se utilizan para acceder a las bases de datos... 
- Escrito en Python puro y no tiene dependencia excepto la Python Standard Library.

[Descargar MySQL connector](https://dev.mysql.com/downloads/connector/python/)

[Instalar MySQL connector mediante pip](https://pypi.org/project/mysql-connector-python/)

### Conexión con el servidor

El constructor ```connect()``` crea una conexión al servidor MySQL y retorna un objeto ```MySQLConnection```.

- [Argumentos de la conexión](https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html).

Para tratar los errores de conexión, es posible utilizar la sentencia ```try-except``` y detectar todos los errores utilizando las excepciones [```errors.Error```](https://dev.mysql.com/doc/connector-python/en/connector-python-api-errors-error.html).

En el siguiente ejemplo, se realiza una conexión al servidor local (`localhost`) y a la base de datos `universidad`. Si la conexión se logra establecer, entonces, se procede a cerrar dicha conexión.

In [ ]:
from mysql.connector import connect
from mysql.connector import Error
from mysql.connector import errorcode

try:
    cnx = connect(user='root', password='mysqlroot', host='localhost', database='universidad')
    print('Conexión establecida con exito!')

    # logica SQL

    cnx.close()
    print('Conexión cerrada!')

except Error as e:
    if e.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('Algo anda mal con tu usuario o contraseña.')
    elif e.errno == errorcode.ER_BAD_DB_ERROR:
        print('La base de datos no existe.')
    else:
        print(e)

### Consultar datos

La clase `MySQLCursor` instancia un __objeto que ejecuta operaciones utilizando sentencias `SQL`__, este, interactua con el servidor utilizando un objeto `MySQLConnection`.

Para crear un cursor, se utiliza el método ```cursor()``` del objeto ```MySQLConnection```. 
- Por defecto, __el cursor es un objeto iterable de filas como tuplas__.
- Opcionalmente, pasando como argumento, el parámetro ```dictionary=True```, el objeto se convierte en un iterador de diccionarios, que utiliza como claves los nombres de los campos de la base de datos.

Para los parámetros,
- `dictionary=True` retorna filas como diccionario.
- `tuple=True`, retorna filas como tuplas.

In [ ]:
# abre una conección al servidor MySQL y la almacena en la variable cnx
cnx = connect(user='root', password='mysqlroot', host='localhost', database='universidad')

# crea un cursor MySQLCursor
cursor = cnx.cursor()

# prepara la consulta
query = ('SELECT * from alumno')

# ejecuta la consulta y retorna los valores
cursor.execute(query)

for rut, nombre in cursor:
    print('{} : {}'.format(rut, nombre))

cursor.close()
cnx.close()

En el siguiente ejemplo, se ejecuta la calusula [`INNER JOIN`](https://www.w3schools.com/sql/sql_join_inner.asp) para extraer los profesores que actualmente dictan las asignaturas registradas:

In [ ]:
cnx = connect(user='root', password='mysqlroot', host='localhost', database='universidad')

cursor = cnx.cursor()

query = ("""SELECT asignatura.nombre, profesor.nombre
            FROM asignatura INNER JOIN profesor ON asignatura.rutProf = profesor.rutProf""")

# ejecuta la consulta y retorna los valores
cursor.execute(query)

for asignatura, profesor in cursor:
    espacio = profesor.index(' ')
    nombre = profesor[:espacio]
    apellido = profesor[espacio+1:]
    print('{:11}: {} {}'.format(asignatura.upper(), nombre.capitalize(), apellido.capitalize()))

cursor.close()
cnx.close()

### Crear tablas

Por lo general, las tablas de una base de datos son objetos permanentes y es posible trabajar con tablas ya creadas, en lugar de crearlas. No obstante, sentecias DDL (*Data Definition Language*) se pueden ejecutar a partir de un objetos cursor.

**NOTA: Se recomienda evitar la continua creación de tablas, ya que es una operación costosa.**

En el siguiente ejemplo se crea la base de datos `empleados`.

In [ ]:
DB_NAME = 'employees'
TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE `employees` ("
    "`emp_no` int(11) NOT NULL AUTO_INCREMENT,"
    "`birth_date` date NOT NULL,"
    "`first_name` varchar(14) NOT NULL,"
    "`last_name` varchar(16) NOT NULL,"
    "`gender` enum('M','F') NOT NULL,"
    "`hire_date` date NOT NULL,"
    "PRIMARY KEY (`emp_no`)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE `departments` ("
    "`dept_no` char(4) NOT NULL,"
    "`dept_name` varchar(40) NOT NULL,"
    "PRIMARY KEY (`dept_no`), UNIQUE KEY `dept_name` (`dept_name`)"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE `dept_emp` ("
    "`emp_no` int(11) NOT NULL,"
    "`dept_no` char(4) NOT NULL,"
    "`from_date` date NOT NULL,"
    "`to_date` date NOT NULL,"
    "PRIMARY KEY (`emp_no`,`dept_no`), KEY `emp_no` (`emp_no`),"
    "KEY `dept_no` (`dept_no`),"
    "CONSTRAINT `dept_emp_ibfk_1` FOREIGN KEY (`emp_no`) "
    "REFERENCES `employees` (`emp_no`) ON DELETE CASCADE,"
    "CONSTRAINT `dept_emp_ibfk_2` FOREIGN KEY (`dept_no`) "
    "REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

La función ```create_database(cursor, db)``` que se define a continuación, crea una base de datos a partir de un cursor `MySQLCursor`.
- Recibe como argumentos __el cursor__ que contiene la sentencia `SQL` (DDL) y el __nombre de la base de datos__.
- Incluye manejo de excepciones para la ejecución de lenguaje `SQL`.

In [ ]:
def create_database(cursor, db_name):
    try:
        cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(db_name))
    except Error as e:
        print('Error al crear la base de datos: {}'.format(e))

En el siguiente código, se emplea la sentencia `try-except` que intenta conectarse a la base de datos de nombre `DB_NAME`. 
- Es el caso que suceda un error de código [`ER_BAD_DB_ERROR`](https://dev.mysql.com/doc/refman/5.7/en/error-messages-server.html#error_er_bad_db_error) (`ERROR 1049 (42000): Unknown database`), es decir, que no existe, se ejecuta la función `create_database()` que crea la dabe de datos.

In [ ]:
cnx = connect(user='root', password='mysqlroot', host='localhost')
cursor = cnx.cursor()

# conectándose a la base de datos
try:
    cursor.execute("USE {}".format(DB_NAME))
    print("Base de datos {} seleccionada.".format(DB_NAME))
except Error as e:
    if e.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos {} no existe.".format(DB_NAME))
        create_database(cursor, DB_NAME)
        print("Base de datos {} fue creada exitosamente.".format(DB_NAME))
    elif e.errno == errorcode.ER_DB_EXISTS_ERROR:
        print('ya existe.')
    else:
        print(e)

Una vez que se crea la base de datos, o si existe, se cambia a la base de datos objetivo, se crean las tablas iterando sobre los items del diccionario `TABLES`.

In [ ]:
cursor.execute("USE {}".format(DB_NAME))
for tablename, ddlcode in TABLES.items():
    try:
        print('Creando tabla {}: '.format(tablename), end='')
        cursor.execute(ddlcode)
        print('OK')
    except Error as e:
        if e.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print('Tabla {} ya existe.'.format(tablename))
        else:
            print(e.msg)

### Insertar datos

En el siguiente ejemplo se insertan nuevos datos en la tabla `employees`.

In [ ]:
from datetime import date, datetime, timedelta

cnx = connect(user='root', password='mysqlroot', host='localhost', database='employees')
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)
add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
data_employee = ('Juan', 'Perez', tomorrow, 'M', date(1977, 6, 14))

# insertar nuevo registro
cursor.execute(add_employee, data_employee)

# asegurarse que los datos estén ingresados en la base de datos
cnx.commit()

cursor.close()
cnx.close()

## Actividad

1. Diseñe la función <code>conexionMySQL()</code> que reciba como argumento, el usuario y su contraseña, el <em>host</em> y el nombre de la base de datos. La función debe devolver el objeto <code>MySQLConnection</code>, si la conexión fue realizada con éxito, de lo contrario, debe devolver el valor <code>None</code>.
2. A partir de la base de datos diseñada para la gestión de instrumento del laboratorio de geodesia y topografía del DCGG, generar 3 funciones de Python para abordar los siguientes requerimientos,
    - Realizar un préstamo (o reserva) de instrumento
    - Registrar la devolución de instrumentos
    - Verificar el estado de un instrumento, i.e., si esta disponible para préstamo.

## Actividades complementarias

1. Investigar la extracción de datos desde una base de datos en MySQL desde Microsoft Excel.
2. Investigar el uso de la librería Python [pyodbc](https://pypi.org/project/pyodbc/#description).